In [1]:
from pyspark import SparkConf, SparkContext
import math, sys, glob, random
import sympy
import numpy as np
from nltk import ngrams, word_tokenize
import json
import matplotlib.pyplot as plt
plt.style.use('ggplot')

conf = SparkConf().setMaster("local").setAppName("Final")
sc = SparkContext(conf=conf)

In [2]:
with open('./data/Gossiping_page_1_11.json') as f:
    gossip = json.load(f)

In [3]:
gossip[0].keys()

dict_keys(['board', 'article_id', 'article_title', 'author', 'date', 'content', 'ip', 'message_count', 'messages', 'error'])

In [4]:
gossip[0]['messages']

[{'push_tag': '推',
  'push_userid': 'kirai',
  'push_content': '看來美國也是那種打著自由民主平等卻還是歧視少數ꐠ',
  'push_ipdatetime': '61.224.135.28 07/05'},
 {'push_tag': '→',
  'push_userid': 'kirai',
  'push_content': '族群的國家...',
  'push_ipdatetime': '61.224.135.28 07/05'},
 {'push_tag': '→',
  'push_userid': 'loveangler',
  'push_content': '沒辦法,某些運動界還是很保守的,像是棒球籃球',
  'push_ipdatetime': '61.231.82.117 07/05'},
 {'push_tag': '→',
  'push_userid': 'loveangler',
  'push_content': '他們還是不太能接受同志,因為球員跟球員之間',
  'push_ipdatetime': '61.231.82.117 07/05'},
 {'push_tag': '→',
  'push_userid': 'loveangler',
  'push_content': '常會有脂體碰撞,或是袒程相見之狀況,一些不了',
  'push_ipdatetime': '61.231.82.117 07/05'},
 {'push_tag': '→',
  'push_userid': 'loveangler',
  'push_content': '解同志的,或是對同志存有偏見的,就會覺得那樣',
  'push_ipdatetime': '61.231.82.117 07/05'},
 {'push_tag': '→',
  'push_userid': 'loveangler',
  'push_content': '很怪不自在,只能希望下一代這種偏見會越來越',
  'push_ipdatetime': '61.231.82.117 07/05'},
 {'push_tag': '→',
  'push_userid': 'loveangler',
 

## Graph Construction

In [5]:
all_user = {}
tag = {'推':1, '噓':-1, '→':0}

### comment : 1：贊, 0：沒情緒, -1：噓
# Mapper : (Author, [(push_user, comment), (push_user2, comment), ...])
def load_users(board):
    result = []
    for artical in board:
        if(artical['author'] not in all_user):
            all_user[artical['author']] = len(all_user)
        
        author = all_user[artical['author']]
        
        pusher_comment = []
        for push_artical in artical['messages']:
            if(push_artical['push_userid'] not in all_user):
                all_user[push_artical['push_userid']] = len(all_user)
            pusher_comment.append((all_user[push_artical['push_userid']], tag[push_artical['push_tag']]))
            
        result.append((author, pusher_comment))
    return result        

In [6]:
gossip_graph = sc.parallelize(load_users(gossip))

In [7]:
gossip_graph.collect()[0]

(0,
 [(1, 1),
  (1, 0),
  (2, 0),
  (2, 0),
  (2, 0),
  (2, 0),
  (2, 0),
  (2, 0),
  (2, 1),
  (2, 0),
  (2, 0),
  (2, 0),
  (3, 1),
  (4, 0),
  (4, 0),
  (5, -1),
  (6, 0),
  (6, 0),
  (7, 1)])

In [8]:
g = gossip_graph.flatMap(lambda x: [(x[0], (xx[0], xx[1])) for xx in x[1]])

In [9]:
g.collect()[22]

(8, (12, 1))

In [10]:
from operator import add
talk = g.map(lambda x: ((x[0], x[1][0]), x[1][1]))
talk2 = talk.reduceByKey(add)

In [11]:
len(talk.collect())

10731

In [12]:
len(talk2.collect())

6901

In [13]:
len(talk2.filter(lambda x: x[1] < 0).collect())

974